# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime

In [3]:
def format_delta(delta: datetime.timedelta):
    hours, remainder = divmod(delta.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    h = f'{int(hours)}h ' if hours > 0 else ''
    m = f'{int(minutes)}m ' if minutes > 0 else ''
    s = f'{int(seconds)}s'
    
    return h+m+s

def estimate_time_to_finish(times: list, current_run_i: int, total_runs: int):
    deltas = []
    i = 1
    
    while i+1 <= len(times):
        deltas.append(times[i] - times[i-1])
        i += 1
        
    avg = sum(deltas, datetime.timedelta())/len(deltas)
    return f'Run {current_run_i}/{total_runs}. {format_delta(avg)} per run, estimate {format_delta(avg*(total_runs - current_run_i))} to finish'

In [4]:
letter = {"name": "letter",
         "path": "../datasets/letter/dirty.csv",
         "clean_path": "../datasets/letter/clean.csv"}

adult = {
    "name": "adult",
    "path": "../datasets/adult/dirty.csv",
    "clean_path": "../datasets/adult/clean.csv"
}
beers = {
    "name": "beers",
    "path": "../datasets/beers/dirty.csv",
    "clean_path": "../datasets/beers/clean.csv"
}
flights = {
    "name": "flights",
    "path": "../datasets/flights/dirty.csv",
    "clean_path": "../datasets/flights/clean.csv"
}
movies = {
    "name": "movies",
    "path": "../datasets/movies_1/dirty.csv",
    "clean_path": "../datasets/movies_1/clean.csv"
}
rayyan = {
    "name": "rayyan",
    "path": "../datasets/rayyan/dirty.csv",
    "clean_path": "../datasets/rayyan/clean.csv"
}
tax = {
    "name": "tax",
    "path": "../datasets/tax/dirty.csv",
    "clean_path": "../datasets/tax/clean.csv"
}
toy = {
    "name": "toy",
    "path": "../datasets/toy/dirty.csv",
    "clean_path": "../datasets/toy/clean.csv"
}
hospital = {
    "name": "hospital",
    "path": "../datasets/hospital/dirty.csv",
    "clean_path": "../datasets/hospital/clean.csv"
}

In [8]:
results = []

feature_generator = ['value', 'domain', 'vicinity']
vicinity_feature_generator = 'pdep'
vicinity_orders_sets = [[1], [2], [1, 2]]
datasets = [rayyan, beers, flights, hospital, letter]
gpdep_correction_thresholds = [0, 0.03, 0.05]
runs = range(3)

duration_experiment = len(feature_generators_sets) * len(vicinity_feature_generators) * len(vicinity_orders_sets) * len(datasets) * len(runs)
times = [datetime.datetime.now()]
i = 1

for run in runs:
    for vicinity_order in vicinity_orders_sets:
        for dataset in datasets:
            for t in gpdep_correction_thresholds:
                data = raha.Dataset(dataset)
                data.detected_cells = dict(data.get_actual_errors_dictionary())
                app = raha.Correction()
                app.LABELING_BUDGET = 20
                app.VERBOSE = False
                app.FEATURE_GENERATORS = feature_generator
                app.CLASSIFICATION_MODEL = "ABC"
                app.VICINITY_ORDERS = vicinity_order
                app.VICINITY_FEATURE_GENERATOR = vicinity_feature_generator
                app.N_BEST_PDEPS = 5
                app.GPDEP_CORRECTION_SCORE_THRESHOLD = t


                d = app.initialize_dataset(data)
                app.initialize_models(d)

                while len(d.labeled_tuples) < app.LABELING_BUDGET:
                    app.sample_tuple(d, random_seed=None)
                    app.label_with_ground_truth(d)
                    app.update_models(d)
                    app.generate_features(d, synchronous=True)
                    app.predict_corrections(d, random_seed=None)

                p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
                result = {'dataset': d.name, 
                          'vicinity_order': vicinity_order,
                          'gpdep_correction_threshold': t,
                          'run': run,
                          'precision': p, 'recall': r, 'f1': f}                    
                results.append(result)
                print(result)

                # time estimates for the measurement
                times.append(datetime.datetime.now())
                print(estimate_time_to_finish(times, i, duration_experiment))
                i += 1

{'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0, 'run': 0, 'precision': 0.8030050083472454, 'recall': 0.5073839662447257, 'f1': 0.6218487394957982}
Run 1/45. 26s per run, estimate 19m 6s to finish
{'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.03, 'run': 0, 'precision': 0.9396092362344582, 'recall': 0.5580168776371308, 'f1': 0.700198544010589}
Run 2/45. 25s per run, estimate 18m 1s to finish
{'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.05, 'run': 0, 'precision': 0.3320754716981132, 'recall': 0.27848101265822783, 'f1': 0.3029259896729776}
Run 3/45. 24s per run, estimate 17m 20s to finish
{'dataset': 'beers', 'vicinity_order': [1], 'gpdep_correction_threshold': 0, 'run': 0, 'precision': 0.859697386519945, 'recall': 0.859697386519945, 'f1': 0.859697386519945}
Run 4/45. 29s per run, estimate 20m 4s to finish
{'dataset': 'beers', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.03, 'run': 0, 'pr

In [9]:
print(results)

[{'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0, 'run': 0, 'precision': 0.8030050083472454, 'recall': 0.5073839662447257, 'f1': 0.6218487394957982}, {'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.03, 'run': 0, 'precision': 0.9396092362344582, 'recall': 0.5580168776371308, 'f1': 0.700198544010589}, {'dataset': 'rayyan', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.05, 'run': 0, 'precision': 0.3320754716981132, 'recall': 0.27848101265822783, 'f1': 0.3029259896729776}, {'dataset': 'beers', 'vicinity_order': [1], 'gpdep_correction_threshold': 0, 'run': 0, 'precision': 0.859697386519945, 'recall': 0.859697386519945, 'f1': 0.859697386519945}, {'dataset': 'beers', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.03, 'run': 0, 'precision': 0.9690366972477065, 'recall': 0.9685923888124713, 'f1': 0.9688144920889704}, {'dataset': 'beers', 'vicinity_order': [1], 'gpdep_correction_threshold': 0.05, 'run': 0, 'precision': 0.